In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/ComputerVision/Covid19-Detector/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/ComputerVision/Covid19-Detector


In [ ]:
!pip install import-ipynb
import import_ipynb

In [ ]:
%ls

data_loader.ipynb               neural_network_architecture.ipynb
datasets/                       nn/
imagetoarraypreprocessor.ipynb  preprocessing/
main_file.ipynb                 __pycache__/
metadata.csv                    simplepreprocessor.ipynb
metadataFilter.ipynb


Load Dependecies

In [ ]:
from data_loader import DatasetLoader
from simplepreprocessor import SimplePreprocessor
from imagetoarraypreprocessor import ImageToArrayPreprocessor
from neural_network_architecture import  LeNet, MiniVGGNet, ShallowNet

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt
import numpy as np

In [ ]:

WIDTH = 224
HEIGHT = 224

sp = SimplePreprocessor(WIDTH, HEIGHT)
iap = ImageToArrayPreprocessor()

In [ ]:
dl = DatasetLoader(preprocessors=[sp, iap])

In [ ]:
(data, labels) = dl.load_data()

In [ ]:
import pandas as pd
pd.DataFrame(labels, columns = ['labels'])['labels'].value_counts()

Negative    5216
positive     113
Name: labels, dtype: int64

Partition the data into training and testing splits using 75% of the data for
training and the remaining 25% for testing

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.25, random_state=412)

In [ ]:
pd.DataFrame(testY, columns = ['labels'])['labels'].value_counts()

Negative    1312
positive      21
Name: labels, dtype: int64

In [ ]:
# convert the labels from integers to vectors
#trainY = LabelBinarizer().fit_transform(trainY)
#testY = LabelBinarizer().fit_transform(testY)

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
le = LabelEncoder().fit(trainY)
trainY = to_categorical(le.transform(trainY), 2)
testY = to_categorical(le.transform(testY), 2)

In [ ]:
trainY

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

Augmenting Training Dataset for Imabalance Class Handling

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# initialize an our data augmenter 
aug = ImageDataGenerator(
		rotation_range=20,
		zoom_range=0.15,
		width_shift_range=0.2,
		height_shift_range=0.2,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

In [ ]:
print("Compiling Shallow Net Model")


INIT_LR = 1e-3
BS = 32
EPOCHS = 100
opt = SGD(lr=INIT_LR, momentum=0.9, decay=INIT_LR / EPOCHS)
model = ShallowNet.build(width=WIDTH, height=HEIGHT, depth=3, classes=2)
model.compile(loss= "binary_crossentropy", optimizer=opt,
metrics=["accuracy"])

# train the network
print("[INFO] training network...")
H = model.fit(x=aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS)


Compiling Shallow Net Model
[INFO] training network...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/100
124/124 [==============================] - 39s 310ms/step - loss: 0.2213 - accuracy: 0.9770 - val_loss: 0.0319 - val_accuracy: 0.9895
Epoch 2/100
124/124 [==============================] - 38s 305ms/step - loss: 0.0547 - accuracy: 0.9826 - val_loss: 0.0306 - val_accuracy: 0.9887
Epoch 3/100
124/124 [==============================] - 37s 300ms/step - loss: 0.0552 - accuracy: 0.9826 - val_loss: 0.0290 - val_accuracy: 0.9895
Epoch 4/100
124/124 [==============================] - 37s 296ms/step - loss: 0.0582 - accuracy: 0.9813 - val_loss: 0.0300 - val_accuracy: 0.9880
Epoch 5/100
124/124 [==============================] - 37s 295ms/step - loss: 0.0654 - accuracy: 0.9808 - val_loss: 0.0287 - val_accuracy: 0.9880
Epoch 6/100
124/124 [==============================] - 37s 295ms/step - loss: 0.0603 - accuracy: 0.9806 - val_loss: 0.0290 - val_accuracy: 0.9872
Epoch 7/100
124/124 [==============================] - 36s 294ms/step - loss: 0.0514 - accuracy: 0.9846 - val_loss: 0.0370 -

Testing ShallowNet

In [ ]:
# print("[INFO] evaluating network...")
# predictions = model.predict(testX, batch_size=BS)
# print(classification_report(testY.argmax(axis=1),
# predictions.argmax(axis=1), target_names=le.classes_))

[INFO] evaluating network...
              precision    recall  f1-score   support

    Negative       0.99      1.00      1.00      1312
    positive       0.78      0.67      0.72        21

    accuracy                           0.99      1333
   macro avg       0.89      0.83      0.86      1333
weighted avg       0.99      0.99      0.99      1333

